[Reference](https://medium.com/@francescofranco_39234/guide-to-compressing-machine-learning-models-0adc83d1296d)

In [1]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 20.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 498kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.64MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.0MB/s]


In [3]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Knowledge Distillation
## Architecture of the Teacher Model

In [4]:
class TeacherNet(nn.Module):
    def __init__(self):
        super(TeacherNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)  # 32 convolutional filters 5x5
        self.pool = nn.MaxPool2d(5, 5)     # Max pooling 5x5
        self.fc1 = nn.Linear(32 * 4 * 4, 128)  # Layer fully connected
        self.fc2 = nn.Linear(128, 10)           # Output layer (10 classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
teacher_model = TeacherNet()
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.001)
teacher_criterion = nn.CrossEntropyLoss()

for epoch in range(5):
    teacher_model.train()
    running_loss = 0.0

    for data in trainloader:
        inputs, labels = data
        teacher_optimizer.zero_grad()
        outputs = teacher_model(inputs)
        loss = teacher_criterion(outputs, labels)
        loss.backward()
        teacher_optimizer.step()
        running_loss += loss.item()

    teacher_accuracy = evaluate(teacher_model)
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}, "
          f"Accuracy: {teacher_accuracy * 100:.2f}%")

Epoch 1, Loss: 0.21454584467183077, Accuracy: 97.39%
Epoch 2, Loss: 0.07594661512521347, Accuracy: 98.05%
Epoch 3, Loss: 0.05838590213492302, Accuracy: 98.26%
Epoch 4, Loss: 0.04723572207931012, Accuracy: 97.90%
Epoch 5, Loss: 0.04104774282227006, Accuracy: 98.28%


## Architecture of the Student Model

In [6]:
class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Input directed by the flattened images
        self.fc2 = nn.Linear(128, 10)        # Output layer

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the immage 28x28 in vector 784
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training with Knowledge Distillation

In [7]:
def knowledge_distillation_loss(student_logits, teacher_logits):
    p_teacher = F.softmax(teacher_logits, dim=1)      # Teacher probability
    p_student = F.log_softmax(student_logits, dim=1)  # Student Log-probability
    loss = F.kl_div(p_student, p_teacher, reduction='batchmean')
    return loss

In [8]:
student_model = StudentNet()
student_optimizer = optim.Adam(student_model.parameters(), lr=0.001)

for epoch in range(5):
    student_model.train()
    running_loss = 0.0

    for data in trainloader:
        inputs, labels = data
        student_optimizer.zero_grad()
        student_logits = student_model(inputs)
        # Detach to avoid backpropagation via the teacher
        teacher_logits = teacher_model(inputs).detach()
        loss = knowledge_distillation_loss(student_logits, teacher_logits)
        loss.backward()
        student_optimizer.step()
        running_loss += loss.item()

    student_accuracy = evaluate(student_model)
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(testloader)}, "
          f"Accuracy: {student_accuracy * 100:.2f}%")

Epoch 1, Loss: 1.3223543562540772, Accuracy: 95.90%
Epoch 2, Loss: 0.5144708617620027, Accuracy: 97.08%
Epoch 3, Loss: 0.34498830446914125, Accuracy: 97.33%
Epoch 4, Loss: 0.27934224355764165, Accuracy: 97.34%
Epoch 5, Loss: 0.21892536989899625, Accuracy: 97.64%
